In [195]:
import numpy as np
import pandas as pd
import math
from functools import reduce

In [196]:
livetimes_ds3518 = pd.read_csv("livetimes_ds3518.dat", delimiter="\t", names = ["Channel", "Livetime_ds3518"])
livetimes_ds3021 = pd.read_csv("livetimes_ds3021.dat", delimiter="\t", names = ["Channel", "Livetime_ds3021"])
livetimes_ds3051 = pd.read_csv("livetimes_ds3051.dat", delimiter="\t", names = ["Channel", "Livetime_ds3051"])
livetimes_ds3054 = pd.read_csv("livetimes_ds3054.dat", delimiter="\t", names = ["Channel", "Livetime_ds3054"])

In [197]:
livetimes_merged = [livetimes_ds3518, livetimes_ds3021, livetimes_ds3051, livetimes_ds3054]

In [198]:
livetimes_all = reduce(lambda  left,right: pd.merge(left,right,on=['Channel'],
                                            how='outer'), livetimes_merged)
livetimes_all.head()

,Channel,Livetime_ds3518,Livetime_ds3021,Livetime_ds3051,Livetime_ds3054
0,1,1829414.0,2223603.0,3241483.0,3017053.0
1,2,1829414.0,2223603.0,3241483.0,3017053.0
2,3,1829414.0,2223603.0,3241483.0,3017053.0
3,4,1829414.0,2223603.0,0.0,3017053.0
4,5,1829414.0,2165463.0,3241483.0,3017053.0


In [199]:
# Add tower and floor information
# do get_tower before get_floor
def get_tower(row):
    channel = row["Channel"]
    tower = math.ceil(channel/52.0)
    return int(tower)
# requires get_tower first
def get_floor(row):
    channel = row["Channel"]
    tower = row["Tower"]
    channel_in_tower = channel - (52 * (tower - 1))
    floor = channel_in_tower % 13
    if (floor == 0):
        floor = 13
    return int(floor)

In [200]:
livetimes_all["Tower"] = livetimes_all.apply(get_tower, axis=1)
livetimes_all["Floor"] = livetimes_all.apply(get_floor, axis=1)

In [201]:
livetimes_all.head()

,Channel,Livetime_ds3518,Livetime_ds3021,Livetime_ds3051,Livetime_ds3054,Tower,Floor
0,1,1829414.0,2223603.0,3241483.0,3017053.0,1,1
1,2,1829414.0,2223603.0,3241483.0,3017053.0,1,2
2,3,1829414.0,2223603.0,3241483.0,3017053.0,1,3
3,4,1829414.0,2223603.0,0.0,3017053.0,1,4
4,5,1829414.0,2165463.0,3241483.0,3017053.0,1,5


In [208]:
# get livetimes for certain scenarios:

# lifetime of Odd/Even towers
def get_livetime_odd_tower(row):
    tower = int(row["Tower"])
    livetime = []
    if ((tower % 2) == 1):
        livetime.append(row["Livetime_ds3518"])
        livetime.append(row["Livetime_ds3021"])
        livetime.append(row["Livetime_ds3051"])
        livetime.append(row["Livetime_ds3054"])
        return sum(livetime)
    else:
        return 0
    
def get_livetime_even_tower(row):
    tower = int(row["Tower"])
    livetime = []
    if ((tower % 2) == 0):
        livetime.append(row["Livetime_ds3518"])
        livetime.append(row["Livetime_ds3021"])
        livetime.append(row["Livetime_ds3051"])
        livetime.append(row["Livetime_ds3054"])
        return sum(livetime)
    else:
        return 0
    
# Do the same for the floors
def get_livetime_odd_floor(row):
    floor = int(row["Floor"])
    livetime = []
    if ((floor % 2) == 1):
        livetime.append(row["Livetime_ds3518"])
        livetime.append(row["Livetime_ds3021"])
        livetime.append(row["Livetime_ds3051"])
        livetime.append(row["Livetime_ds3054"])
        return sum(livetime)
    else:
        return 0
    
def get_livetime_even_floor(row):
    floor = int(row["Floor"])
    livetime = []
    if ((floor % 2) == 0):
        livetime.append(row["Livetime_ds3518"])
        livetime.append(row["Livetime_ds3021"])
        livetime.append(row["Livetime_ds3051"])
        livetime.append(row["Livetime_ds3054"])
        return sum(livetime)
    else:
        return 0
#lifetime w/o Tower 12
def get_livetime_no_t12(row):
    tower = row["Tower"]
    livetime = []
    if (tower != 12):
        livetime.append(row["Livetime_ds3518"])
        livetime.append(row["Livetime_ds3021"])
        livetime.append(row["Livetime_ds3051"])
        livetime.append(row["Livetime_ds3054"])
        return sum(livetime)
    else:
        return 0

In [209]:
livetimes_all["Odd_Tower"] = livetimes_all.apply(get_livetime_odd_tower, axis=1)
livetimes_all["Even_Tower"] = livetimes_all.apply(get_livetime_even_tower, axis=1)
livetimes_all["Odd_Floor"] = livetimes_all.apply(get_livetime_odd_floor, axis=1)
livetimes_all["Even_Floor"] = livetimes_all.apply(get_livetime_even_floor, axis=1)
livetimes_all["No_T12"] = livetimes_all.apply(get_livetime_no_t12, axis=1)
livetimes_all.head()

,Channel,Livetime_ds3518,Livetime_ds3021,Livetime_ds3051,Livetime_ds3054,Tower,Floor,Odd_Tower,Even_Tower,Odd_Floor,Even_Floor,No_T12
0,1,1829414.0,2223603.0,3241483.0,3017053.0,1,1,10311553.0,0.0,10311553.0,0.0,10311553.0
1,2,1829414.0,2223603.0,3241483.0,3017053.0,1,2,10311553.0,0.0,0.0,10311553.0,10311553.0
2,3,1829414.0,2223603.0,3241483.0,3017053.0,1,3,10311553.0,0.0,10311553.0,0.0,10311553.0
3,4,1829414.0,2223603.0,0.0,3017053.0,1,4,7070070.0,0.0,0.0,7070070.0,7070070.0
4,5,1829414.0,2165463.0,3241483.0,3017053.0,1,5,10253413.0,0.0,10253413.0,0.0,10253413.0


In [210]:
# Turn livetime into exposure
def convert_to_exposure(livetime):
    return livetime * (0.75 / (60 * 60 * 24 * 365.25))

In [212]:
#lifetimes to use in analysis
print ("3018 exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[1])))
print ("3021 exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[2])))
print ("3051 exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[3])))
print ("3054 exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[4])))
print ("Total exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[4] + livetimes_all.sum(axis=0)[3] + livetimes_all.sum(axis=0)[2] + livetimes_all.sum(axis=0)[1])))
print ("Odd Tower exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[7])))
print ("Even Tower exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[8])))
print ("Odd Floor exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[9])))
print ("Even Floor exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[10])))
print ("No T12 exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[11])))


3018 exposure: 36.04995895600426
3021 exposure: 47.34232952600958
3051 exposure: 68.807151779603
3054 exposure: 66.32527369001444
Total exposure: 218.5247139516313
Odd Tower exposure: 117.75819121225949
Even Tower exposure: 100.76652273937181
Odd Floor exposure: 116.26624985740361
Even Floor exposure: 102.25846409422769
No T12 exposure: 208.5950123108221
